In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/working/export.pkl
/kaggle/working/bears/grizzly/f17b530e-f28f-4ec4-93ee-a14fa628dcab.jpg
/kaggle/working/bears/grizzly/8d73a850-6e23-4b2e-8d76-f1626cc562a6.jpg
/kaggle/working/bears/grizzly/cfadc393-0063-4848-9a06-3741ebbceac4.jpg
/kaggle/working/bears/grizzly/0dff2654-692f-47b2-9f6a-d8f1b25578ac.JPG
/kaggle/working/bears/grizzly/ad995546-3b17-4bd8-938b-570594ac00ec.jpg
/kaggle/working/bears/grizzly/8d36c896-6092-41d8-a48e-10fd4263986c.jpg
/kaggle/working/bears/grizzly/b77d3a44-2020-46ed-864f-408b537ebc3d.jpg
/kaggle/working/bears/grizzly/53ab2640-747f-4e75-a90c-c7c9d7005502.jpg
/kaggle/working/bears/grizzly/803227f7-e6ea-4b3a-af9d-1ec84b510824.jpg
/kaggle/working/bears/grizzly/36b1acbc-a1bf-4f2a-9105-6bb13319c112.jpg
/kaggle/working/bears/grizzly/81ead101-cd58-4535-bef4-5ea5c9ed5f46.jpg
/kaggle/working/bears/grizzly/2f9016de-c6e9-471f-8946-b8a924dedb55.jpg
/kaggle/working/bears/grizzly/6d88df8e-4bbf-466c-b1d8-b71eb90b1cac.jpg
/kaggle/working/bears/gr

In [2]:
!pip install -Uqq fastai fastbook

In [3]:
import fastbook
fastbook.setup_book()

In [5]:
from fastbook import *
from fastai.vision.widgets import *

In [6]:
key = os.environ.get('AZURE_SEARCH_KEY', '9fb906081a284f5aaa5c900c8e16515b')

In [15]:
!ls

bears


In [10]:
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print(cwd)

/kaggle/working


In [11]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [12]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [13]:
fns = get_image_files(path)
fns

(#413) [Path('bears/grizzly/f17b530e-f28f-4ec4-93ee-a14fa628dcab.jpg'),Path('bears/grizzly/8d73a850-6e23-4b2e-8d76-f1626cc562a6.jpg'),Path('bears/grizzly/cfadc393-0063-4848-9a06-3741ebbceac4.jpg'),Path('bears/grizzly/0dff2654-692f-47b2-9f6a-d8f1b25578ac.JPG'),Path('bears/grizzly/15c84224-44a6-48e6-bfaf-a55088598566.JPG'),Path('bears/grizzly/ad995546-3b17-4bd8-938b-570594ac00ec.jpg'),Path('bears/grizzly/8d36c896-6092-41d8-a48e-10fd4263986c.jpg'),Path('bears/grizzly/b77d3a44-2020-46ed-864f-408b537ebc3d.jpg'),Path('bears/grizzly/53ab2640-747f-4e75-a90c-c7c9d7005502.jpg'),Path('bears/grizzly/803227f7-e6ea-4b3a-af9d-1ec84b510824.jpg')...]

In [14]:
failed = verify_images(fns)
failed

(#23) [Path('bears/grizzly/15c84224-44a6-48e6-bfaf-a55088598566.JPG'),Path('bears/grizzly/92a6a4cd-cca6-4d8c-b1fe-d9d302965ef1.jpg'),Path('bears/grizzly/7b849320-2f3a-4502-9e2f-cd65c901aca1.jpg'),Path('bears/grizzly/bd8d6027-cc85-4258-a1c6-d4f714f90c33.jpg'),Path('bears/grizzly/50b27505-44f4-411d-9b88-7a924fcd8165.jpg'),Path('bears/black/7db25411-14dd-43a8-9a7f-0580c9e48bd6.jpg'),Path('bears/black/54fc5d52-de3e-4822-819e-34566e657275.jpg'),Path('bears/black/66940a89-a3bf-4fca-8ac5-b2e892fe3ada.jpg'),Path('bears/black/1fbda4a3-17fe-40fc-ae5b-e76cc3ff50a9.jpg'),Path('bears/black/ecb2c804-655b-4ee6-8def-4bf8e320d799.jpg')...]

In [18]:
failed.map(Path.unlink);

In [19]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [20]:
path

Path('bears')

In [21]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [22]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s] 


epoch,train_loss,valid_loss,error_rate,time
0,1.327918,0.101516,0.012821,00:16


epoch,train_loss,valid_loss,error_rate,time
0,0.310822,0.083369,0.012821,00:16
1,0.224639,0.100342,0.012821,00:16
2,0.172837,0.123789,0.025641,00:15
3,0.138948,0.118703,0.025641,00:16


In [23]:
learn.export()

In [24]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [26]:
learn_inf = load_learner(path/'export.pkl')

# **Creating a Notebook App from the Model**

In [35]:
out_pl = widgets.Output()

In [37]:
lbl_pred = widgets.Label()

In [38]:
btn_run = widgets.Button(description='Classify')

In [39]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [40]:
btn_upload = widgets.FileUpload()

In [41]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])